In [12]:
# Celda 1: Importaciones y carga del dataset maestro
import pandas as pd
import numpy as np

# Cargar el dataset maestro de 5 años con datos climáticos y geográficos
try:
    df = pd.read_csv('../data/processed/datos_climaticos_geograficos_nasa_5años.csv', index_col=0)
    df.index = pd.to_datetime(df.index)
    print("Dataset maestro cargado exitosamente.")
    print(f"El dataset contiene {df.shape[0]} registros diarios.")
    display(df.head())
    display(df.info())
except FileNotFoundError:
    print("Error: No se encontró el archivo 'datos_climaticos_geograficos_nasa_5años.csv'.")
    print("Asegúrate de haber ejecutado el notebook 01_exploracion_y_adquisicion.ipynb correctamente.")

Dataset maestro cargado exitosamente.
El dataset contiene 43824 registros diarios.


,ALLSKY_SFC_SW_DWN,WS10M,T2M,PRECTOTCORR,provincia,altitud
2019-01-01,160.59,4.90,12.55,0.14,Azuay,2560
2019-01-02,185.62,5.19,14.09,0.06,Azuay,2560
2019-01-03,148.69,4.35,14.21,0.45,Azuay,2560
2019-01-04,181.20,4.27,13.64,0.92,Azuay,2560
2019-01-05,155.28,3.84,13.63,0.52,Azuay,2560


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2019-01-01 to 2023-12-31
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ALLSKY_SFC_SW_DWN  43824 non-null  float64
 1   WS10M              43824 non-null  float64
 2   T2M                43824 non-null  float64
 3   PRECTOTCORR        43824 non-null  float64
 4   provincia          43824 non-null  object 
 5   altitud            43824 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 2.3+ MB


None

In [13]:
# Celda 2: Constantes físicas y supuestos del sistema fotovoltaico

# Supuestos para un panel solar hipotético
AREA_PANEL_M2 = 2.0      # Área de un panel solar (ej. 2m x 1m)
EFICIENCIA_PANEL = 0.20  # Eficiencia estándar de un panel comercial (20%)
PERDIDAS_SISTEMA = 0.75  # Factor de pérdidas (por cableado, inversor, suciedad, etc.)

# Constantes para la viabilidad
# Estos valores los definiremos más tarde, pero los declaramos aquí
# Se basarán en los cuantiles de la producción de energía
VIABILIDAD_BAJA_KWH = 0
VIABILIDAD_MEDIA_KWH = 0
VIABILIDAD_ALTA_KWH = 0

print("--- Supuestos del Modelo Solar ---")
print(f"Área del panel: {AREA_PANEL_M2} m²")
print(f"Eficiencia del panel: {EFICIENCIA_PANEL * 100}%")
print(f"Factor de pérdidas del sistema: {PERDIDAS_SISTEMA * 100}%")

--- Supuestos del Modelo Solar ---
Área del panel: 2.0 m²
Eficiencia del panel: 20.0%
Factor de pérdidas del sistema: 75.0%


In [14]:
# Celda 3: Cálculo de la energía y potencia diaria potencial

# Energía diaria producida (kWh/día)
# Energía = Radiación (kWh/m²/día) * Área (m²) * Eficiencia * Pérdidas
df['energia_solar_diaria_kwh'] = df['ALLSKY_SFC_SW_DWN'] * AREA_PANEL_M2 * EFICIENCIA_PANEL * PERDIDAS_SISTEMA

# Potencia media diaria (kW)
# Potencia Media = Energía Total del día (kWh) / 24 horas
df['potencia_solar_media_diaria_kw'] = df['energia_solar_diaria_kwh'] / 24

print("--- Cálculo de potencia y energía diaria completado ---")
display(df[['provincia', 'ALLSKY_SFC_SW_DWN', 'energia_solar_diaria_kwh', 'potencia_solar_media_diaria_kw']].head())

--- Cálculo de potencia y energía diaria completado ---


,provincia,ALLSKY_SFC_SW_DWN,energia_solar_diaria_kwh,potencia_solar_media_diaria_kw
2019-01-01,Azuay,160.59,48.177,2.007375
2019-01-02,Azuay,185.62,55.686,2.320250
2019-01-03,Azuay,148.69,44.607,1.858625
2019-01-04,Azuay,181.20,54.360,2.265000
2019-01-05,Azuay,155.28,46.584,1.941000


In [15]:
# Celda 3.1: Constantes y supuestos para energía eólica e hídrica

# --- Energía Eólica ---
# Supuestos para un aerogenerador hipotético
DIAMETRO_ROTOR_M = 90       # Diámetro del rotor (ej. 90m, un tamaño común)
AREA_BARRIDA_M2 = np.pi * (DIAMETRO_ROTOR_M / 2) ** 2
EFICIENCIA_AEROGENERADOR = 0.35 # Eficiencia típica (límite de Betz es 59.3%)
DENSIDAD_AIRE_KG_M3 = 1.225  # Densidad del aire a nivel del mar (kg/m³)
# Nota: La densidad del aire disminuye con la altitud, pero usaremos un valor estándar para simplificar.

# --- Energía Hídrica ---
# Usaremos la precipitación como un proxy del potencial hídrico.
# Es una simplificación, pero un buen punto de partida.
# Un análisis más avanzado usaría datos de caudal de ríos.
print("--- Supuestos para Energía Eólica ---")
print(f"Diámetro del rotor: {DIAMETRO_ROTOR_M} m")
print(f"Área barrida: {AREA_BARRIDA_M2:.2f} m²")
print(f"Eficiencia del aerogenerador: {EFICIENCIA_AEROGENERADOR * 100:.2f}%")
print(f"Densidad del aire (estándar): {DENSIDAD_AIRE_KG_M3} kg/m³")
print("\n--- Supuestos para Energía Hídrica ---")
print("Se usará la precipitación anual total (mm) como proxy del potencial hídrico.")

--- Supuestos para Energía Eólica ---
Diámetro del rotor: 90 m
Área barrida: 6361.73 m²
Eficiencia del aerogenerador: 35.00%
Densidad del aire (estándar): 1.225 kg/m³

--- Supuestos para Energía Hídrica ---
Se usará la precipitación anual total (mm) como proxy del potencial hídrico.


In [16]:
# Celda 3.2: Cálculo de la energía y potencia eólica diaria

# Potencia instantánea teórica (W) - Usamos la velocidad promedio del día
# Nota: Esto es una simplificación. La potencia real debería usar la distribución de velocidades del día.
df['potencia_eolica_instantanea_w'] = 0.5 * DENSIDAD_AIRE_KG_M3 * AREA_BARRIDA_M2 * (df['WS10M']**3) * EFICIENCIA_AEROGENERADOR

# Energía diaria producida (kWh/día) - Asumimos que la potencia instantánea es constante durante 24h
df['energia_eolica_diaria_kwh'] = (df['potencia_eolica_instantanea_w'] * 24) / 1000

print("--- Cálculo de potencia y energía eólica diaria completado ---")
display(df[['provincia', 'WS10M', 'potencia_eolica_instantanea_w', 'energia_eolica_diaria_kwh']].head())

--- Cálculo de potencia y energía eólica diaria completado ---


,provincia,WS10M,potencia_eolica_instantanea_w,energia_eolica_diaria_kwh
2019-01-01,Azuay,4.90,160449.097173,3850.778332
2019-01-02,Azuay,5.19,190656.278318,4575.750680
2019-01-03,Azuay,4.35,112257.872820,2694.188948
2019-01-04,Azuay,4.27,106177.540890,2548.260981
2019-01-05,Azuay,3.84,77222.296117,1853.335107


In [17]:
# Celda 3.3: Cálculo del potencial hídrico (proxy)

# La variable PRECTOTCORR es la precipitación diaria en mm.
# 1 mm de lluvia equivale a 1 litro de agua por metro cuadrado.
# Usaremos la precipitación total anual como nuestro índice de potencial.
# No es necesario un cálculo diario aquí, lo haremos en la agregación.

print("--- Potencial hídrico definido como precipitación total anual ---")
# El cálculo se hará en la celda de agregación.
display(df[['provincia', 'PRECTOTCORR']].head())

--- Potencial hídrico definido como precipitación total anual ---


,provincia,PRECTOTCORR
2019-01-01,Azuay,0.14
2019-01-02,Azuay,0.06
2019-01-03,Azuay,0.45
2019-01-04,Azuay,0.92
2019-01-05,Azuay,0.52


In [18]:
# Celda 4 (Ampliada): Agregación de datos a nivel anual por provincia

# Extraer el año del índice para agrupar
df['año'] = df.index.year

# Definir las columnas a agregar y cómo hacerlo (AHORA INCLUYE EÓLICA E HÍDRICA)
agg_dict = {
    # Variables climáticas
    'T2M': 'mean',
    'WS10M': 'mean',
    'PRECTOTCORR': 'sum',
    'altitud': 'first',
    
    # Variables objetivo de ENERGÍA SOLAR
    'energia_solar_diaria_kwh': 'sum',
    'potencia_solar_media_diaria_kw': 'mean',
    
    # Variables objetivo de ENERGÍA EÓLICA
    'energia_eolica_diaria_kwh': 'sum',
    'potencia_eolica_instantanea_w': 'mean',
    
    # Variable objetivo de ENERGÍA HÍDRICA (proxy)
    'PRECTOTCORR': 'sum' # La sumamos de nuevo para dejarla clara como variable hídrica
}

# Agrupar por provincia y año y aplicar las agregaciones
df_anual = df.groupby(['provincia', 'año']).agg(agg_dict).reset_index()

# Renombrar columnas para mayor claridad
df_anual.rename(columns={
    'T2M': 'temp_promedio_anual_C',
    'WS10M': 'viento_promedio_anual_ms',
    'PRECTOTCORR': 'potencial_hidrico_proxy_mm', # Renombramos para claridad
    'energia_solar_diaria_kwh': 'energia_solar_anual_kwh',
    'potencia_solar_media_diaria_kw': 'potencia_solar_media_anual_kw',
    'energia_eolica_diaria_kwh': 'energia_eolica_anual_kwh',
    'potencia_eolica_instantanea_w': 'potencia_eolica_media_anual_w'
}, inplace=True)

print("--- Datos agregados a nivel anual (incluyendo Eólica e Hídrica) ---")
print(f"El nuevo dataset tiene {df_anual.shape[0]} registros (provincias x años).")
display(df_anual.head())

--- Datos agregados a nivel anual (incluyendo Eólica e Hídrica) ---
El nuevo dataset tiene 120 registros (provincias x años).


,provincia,año,temp_promedio_anual_C,viento_promedio_anual_ms,potencial_hidrico_proxy_mm,altitud,energia_solar_anual_kwh,potencia_solar_media_anual_kw,energia_eolica_anual_kwh,potencia_eolica_media_anual_w
0,Azuay,2019,13.130822,3.323616,1077.24,2560,17341.146,1.979583,599484.772674,68434.334780
1,Azuay,2020,13.589317,3.194399,860.99,2560,18172.824,2.068855,526072.962378,59889.909196
2,Azuay,2021,13.016658,3.100274,1160.65,2560,17442.852,1.991193,480702.577504,54874.723459
3,Azuay,2022,13.055562,3.035233,952.06,2560,16870.398,1.925845,456863.821097,52153.404235
4,Azuay,2023,13.499918,3.028822,1072.91,2560,17505.078,1.998297,446871.391565,51012.715932


In [19]:
# Celda 5: Creación de la variable de clasificación 'viabilidad_solar'

# Calcular los cuantiles para definir los umbrales
# Esto divide los datos en 3 grupos con igual número de observaciones
umbral_bajo = df_anual['energia_solar_anual_kwh'].quantile(0.33)
umbral_alto = df_anual['energia_solar_anual_kwh'].quantile(0.66)

print(f"Umbral para viabilidad 'Bajo/Medio': {umbral_bajo:.2f} kWh/año")
print(f"Umbral para viabilidad 'Medio/Alto': {umbral_alto:.2f} kWh/año")

# Crear la columna de viabilidad usando pd.cut
bins = [-np.inf, umbral_bajo, umbral_alto, np.inf]
labels = ['Bajo', 'Medio', 'Alto']
df_anual['viabilidad_solar'] = pd.cut(df_anual['energia_solar_anual_kwh'], bins=bins, labels=labels, right=True)

print("\n--- Variable de viabilidad creada ---")
# Contar cuántas provincias caen en cada categoría
print(df_anual['viabilidad_solar'].value_counts())
display(df_anual[['provincia', 'año', 'energia_solar_anual_kwh', 'viabilidad_solar']].head(10))

Umbral para viabilidad 'Bajo/Medio': 17505.08 kWh/año
Umbral para viabilidad 'Medio/Alto': 18146.90 kWh/año

--- Variable de viabilidad creada ---
viabilidad_solar
Bajo     41
Alto     41
Medio    38
Name: count, dtype: int64


,provincia,año,energia_solar_anual_kwh,viabilidad_solar
0,Azuay,2019,17341.146,Bajo
1,Azuay,2020,18172.824,Alto
2,Azuay,2021,17442.852,Bajo
3,Azuay,2022,16870.398,Bajo
4,Azuay,2023,17505.078,Bajo
5,Bolívar,2019,17515.131,Medio
6,Bolívar,2020,18187.422,Alto
7,Bolívar,2021,17228.292,Bajo
8,Bolívar,2022,16653.732,Bajo
9,Bolívar,2023,17730.804,Medio


In [20]:
# Celda 5.1: Creación de variables de viabilidad para Eólica e Hídrica

# --- Viabilidad Eólica ---
umbral_bajo_eolico = df_anual['energia_eolica_anual_kwh'].quantile(0.33)
umbral_alto_eolico = df_anual['energia_eolica_anual_kwh'].quantile(0.66)

bins_eolico = [-np.inf, umbral_bajo_eolico, umbral_alto_eolico, np.inf]
labels = ['Bajo', 'Medio', 'Alto']
df_anual['viabilidad_eolica'] = pd.cut(df_anual['energia_eolica_anual_kwh'], bins=bins_eolico, labels=labels, right=True)

# --- Viabilidad Hídrica ---
umbral_bajo_hidrico = df_anual['potencial_hidrico_proxy_mm'].quantile(0.33)
umbral_alto_hidrico = df_anual['potencial_hidrico_proxy_mm'].quantile(0.66)

bins_hidrico = [-np.inf, umbral_bajo_hidrico, umbral_alto_hidrico, np.inf]
df_anual['viabilidad_hidrica'] = pd.cut(df_anual['potencial_hidrico_proxy_mm'], bins=bins_hidrico, labels=labels, right=True)

print("--- Variables de viabilidad eólica e hídrica creadas ---")
print("\nViabilidad Eólica:")
print(df_anual['viabilidad_eolica'].value_counts())
print("\nViabilidad Hídrica:")
print(df_anual['viabilidad_hidrica'].value_counts())
display(df_anual[['provincia', 'año', 'energia_eolica_anual_kwh', 'viabilidad_eolica', 'potencial_hidrico_proxy_mm', 'viabilidad_hidrica']].head(10))

--- Variables de viabilidad eólica e hídrica creadas ---

Viabilidad Eólica:
viabilidad_eolica
Alto     41
Bajo     40
Medio    39
Name: count, dtype: int64

Viabilidad Hídrica:
viabilidad_hidrica
Alto     41
Bajo     40
Medio    39
Name: count, dtype: int64


,provincia,año,energia_eolica_anual_kwh,viabilidad_eolica,potencial_hidrico_proxy_mm,viabilidad_hidrica
0,Azuay,2019,599484.772674,Alto,1077.24,Medio
1,Azuay,2020,526072.962378,Alto,860.99,Bajo
2,Azuay,2021,480702.577504,Alto,1160.65,Medio
3,Azuay,2022,456863.821097,Medio,952.06,Bajo
4,Azuay,2023,446871.391565,Medio,1072.91,Medio
5,Bolívar,2019,696462.240249,Alto,1565.42,Alto
6,Bolívar,2020,488054.148106,Alto,1211.61,Medio
7,Bolívar,2021,553457.535236,Alto,1782.04,Alto
8,Bolívar,2022,546266.886967,Alto,1620.55,Alto
9,Bolívar,2023,564307.379650,Alto,2017.76,Alto


In [22]:
# Celda 6 (Corregida y Ampliada): Guardar el dataset final para TODOS los modelos

# Seleccionar las columnas finales que usaremos para los modelos (Solar, Eólica, Hídrica)
columnas_finales = [
    # Información base
    'provincia', 'año', 'altitud',
    
    # Variables climáticas
    'temp_promedio_anual_C', 'viento_promedio_anual_ms', 'potencial_hidrico_proxy_mm',
    
    # Variables de Energía SOLAR
    'potencia_solar_media_anual_kw', 'energia_solar_anual_kwh', 'viabilidad_solar',
    
    # Variables de Energía EÓLICA
    'potencia_eolica_media_anual_w', 'energia_eolica_anual_kwh', 'viabilidad_eolica',
    
    # Variables de Energía HÍDRICA
    'viabilidad_hidrica'
]

# Crear el dataframe final con todas las variables
dataset_modelo_energias = df_anual[columnas_finales]

# Guardar en la carpeta de datos procesados con un nuevo nombre
output_path = '../data/processed/dataset_modelo_energias_renovables.csv'
dataset_modelo_energias.to_csv(output_path, index=False)

print(f"--- Dataset final para el modelado de las 3 energías guardado en '{output_path}' ---")
display(dataset_modelo_energias.head())

--- Dataset final para el modelado de las 3 energías guardado en '../data/processed/dataset_modelo_energias_renovables.csv' ---


,provincia,año,altitud,temp_promedio_anual_C,viento_promedio_anual_ms,potencial_hidrico_proxy_mm,potencia_solar_media_anual_kw,energia_solar_anual_kwh,viabilidad_solar,potencia_eolica_media_anual_w,energia_eolica_anual_kwh,viabilidad_eolica,viabilidad_hidrica
0,Azuay,2019,2560,13.130822,3.323616,1077.24,1.979583,17341.146,Bajo,68434.334780,599484.772674,Alto,Medio
1,Azuay,2020,2560,13.589317,3.194399,860.99,2.068855,18172.824,Alto,59889.909196,526072.962378,Alto,Bajo
2,Azuay,2021,2560,13.016658,3.100274,1160.65,1.991193,17442.852,Bajo,54874.723459,480702.577504,Alto,Medio
3,Azuay,2022,2560,13.055562,3.035233,952.06,1.925845,16870.398,Bajo,52153.404235,456863.821097,Medio,Bajo
4,Azuay,2023,2560,13.499918,3.028822,1072.91,1.998297,17505.078,Bajo,51012.715932,446871.391565,Medio,Medio
